In [1]:
import pandas as pd
import numpy as np
import re
import os
from shapely.geometry import Point

In [2]:
msa_states = {
    "New York-Newark-Jersey City, NY-NJ-PA": Point(-73.97735818122587, 40.75360363043727),
    "Los Angeles-Long Beach-Anaheim, CA": Point(-118.24252459246136, 34.05460218452133),
    "Chicago-Naperville-Elgin, IL-IN-WI": Point(-87.63203491924644, 41.884717076254915),
    "Dallas-Fort Worth-Arlington, TX": Point(-96.80279248769227, 32.800965872868694),
    "Houston-The Woodlands-Sugar Land, TX": Point(-95.36937482498811, 29.761089990716613),
    "Washington-Arlington-Alexandria, DC-VA-MD-WV": Point(-77.03652980226985, 38.89785162759491),
    "Miami-Fort Lauderdale-Pompano Beach, FL": Point(-80.23615723041286, 25.73577257017272),
    "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD": Point(-75.16351548874385, 39.953038487867026),
    "Atlanta-Sandy Springs-Alpharetta, GA": Point(-84.390397844751, 33.7488895763707),
    "Phoenix-Mesa-Chandler, AZ": Point(-112.07730967359518, 33.44900148985069),
    "Boston-Cambridge-Newton, MA-NH": Point(-71.05800373099267, 42.3605486787696),
    "San Francisco-Oakland-Berkeley, CA": Point(-122.41924170230465, 37.77946153559538),
    "Riverside-San Bernardino-Ontario, CA": Point(-117.37553547358051, 33.980732883142146),
    "Detroit-Warren-Dearborn, MI": Point(-83.04382800215781, 42.3297768018276),
    "Seattle-Tacoma-Bellevue, WA": Point(-122.32993003265794, 47.60409149522296),
    "Minneapolis-St. Paul-Bloomington, MN-WI": Point(-93.26543833090201, 44.977478004204215),
    "San Diego-Chula Vista-Carlsbad, CA": Point(-117.10765111313297, 32.675450648815755),
    "Tampa-St. Petersburg-Clearwater, FL": Point(-82.45727419909451, 27.952274038205715),
    "Denver-Aurora-Lakewood, CO": Point(-104.99079680224305, 39.73932033012998),
    "St. Louis, MO-IL": Point(-90.19954729857595, 38.62786882898026),
    "Baltimore-Columbia-Towson, MD": Point(-76.6101482594712, 39.29152536367409),
    "Charlotte-Concord-Gastonia, NC-SC": Point(-80.83797995283503, 35.22253748266689),
    "Orlando-Kissimmee-Sanford, FL": Point(-81.37953479478388, 28.538635949167848),
    "San Antonio-New Braunfels, TX": Point(-98.4951415720483, 29.42473466610585),
    "Portland-Vancouver-Hillsboro, OR-WA": Point(-122.67919403198559, 45.515883523179866),
    "Sacramento-Roseville-Folsom, CA": Point(-121.49333111762331, 38.58271161629985),
    "Pittsburgh, PA": Point(-79.99659636546517, 40.44023678221106),
    "Las Vegas-Henderson-Paradise, NV": Point(-115.14855607536857, 36.16747408209834),
    "Austin-Round Rock-Georgetown, TX": Point(-97.74713291785895, 30.265186279240133),
    "Cincinnati, OH-KY-IN": Point(-84.51909611760684, 39.10446774409015),
    "Cleveland-Elyria, OH": Point(-81.6930476140442, 41.5053277050308),
    "Kansas City, MO-KS": Point(-94.578285730115, 39.1010737646118),
    "Columbus, OH": Point(-83.0024547595931, 39.9637007958694),
    "Indianapolis-Carmel-Anderson, IN": Point(-86.1527214971812, 39.7721604823431),
    "San Jose-Sunnyvale-Santa Clara, CA": Point(-121.884315873927, 37.3379980099373),
    "Virginia Beach-Norfolk-Newport News, VA-NC": Point(-76.0558554165705, 36.7529078409642),
    "Nashville-Davidson--Murfreesboro--Franklin, TN": Point(-86.7662065918226, 36.1719947399064),
    "Providence-Warwick, RI-MA": Point(-71.412318525675, 41.8242118708439),
    "Milwaukee-Waukesha, WI": Point(-87.9095751962868, 43.0419137098374),
    "Jacksonville, FL": Point(-81.6609781752069, 30.3756071265681),
    "Memphis, TN-MS-AR": Point(-90.0525006455091, 35.1571305205873),
    "Oklahoma City, OK": Point(-97.5201902178872, 35.4691695406864),
    "Hartford-East Hartford-Middletown, CT": Point(-72.6709678916762, 41.7641327640195),
    "Louisville/Jefferson County, KY-IN": Point(-85.7605710596327, 38.2548592792082),
    "New Orleans-Metairie, LA": Point(-90.0766333463257, 29.9537839928333),
    "Richmond, VA": Point(-77.4317698778892, 37.5414041880906),
    "Buffalo-Cheektowaga, NY": Point(-78.8789065772405, 42.8867608775758),
    "Raleigh-Cary, NC": Point(-78.6428638281766, 35.7788111341718),
    "Salt Lake City, UT": Point(-111.887360818175, 40.7661353706654),
    "Rochester, NY": Point(-77.6141558833138, 43.1571105088485),
    "Birmingham-Hoover, AL": Point(-86.8100188707269, 33.529068145948),
    "Grand Rapids-Kentwood, MI": Point(-85.6710991751044, 42.9757472987403),
    "Tucson, AZ": Point(-110.97295530836, 32.2231612404127),
    "Urban Honolulu, HI": Point(-157.857410100852, 21.3095473693441),
    "Tulsa, OK": Point(-95.9901218815961, 36.1576899024592),
    "Fresno, CA": Point(-119.783643958335, 36.7399589236356),
    "Worcester, MA-CT": Point(-71.8012264542394, 42.2628940064412),
    "Bridgeport-Stamford-Norwalk, CT": Point(-73.1923457029622, 41.1806689168382),
    "Albuquerque, NM": Point(-106.65184220209, 35.0917980651784),
    "Albany-Schenectady-Troy, NY": Point(-73.7543872000024, 42.6519510059115),
    "Omaha-Council Bluffs, NE-IA": Point(-95.9373610092953, 41.2597924597602),
    "New Haven-Milford, CT": Point(-72.9249732759824, 41.3100898957712),
    "Bakersfield, CA": Point(-119.019787756511, 35.3732062493274),
    "Baton Rouge, LA": Point(-91.1890544687199, 30.4554774662898),
    "Greenville-Anderson, SC": Point(-82.3951555110306, 34.8566630853142),
    "Oxnard-Thousand Oaks-Ventura, CA": Point(-119.181614727174, 34.203202717367),
    "Allentown-Bethlehem-Easton, PA-NJ": Point(-75.4667009116915, 40.6040190197423),
    "Knoxville, TN": Point(-83.9158075433303, 35.968663057639),
    "El Paso, TX": Point(-106.484682158435, 31.7619967235255),
    "Dayton-Kettering, OH": Point(-84.1933035680526, 39.7598257315708),
    "McAllen-Edinburg-Mission, TX": Point(-98.2390646772491, 26.301782579367),
    "Columbia, SC": Point(-81.0371440198162, 34.0107137654275),
    "North Port-Sarasota-Bradenton, FL": Point(-82.2069803886827, 27.0754528684216),
    "Charleston-North Charleston, SC": Point(-80.012286644401, 32.8757024464949),
    "San Juan-Bayamón-Caguas, PR": Point(-66.116313035346, 18.4657257515783)
}

msa_states = [msa for msa, value in msa_states.items()]
msa_states = {msa: re.match('.*\,\s([A-Z]{2}).*', msa).group(1) for msa in msa_states}
msa_states = {'cbsa': list(msa_states.keys()), 'state': list(msa_states.values())}
msa_states = pd.DataFrame(msa_states)

In [3]:
df = pd.read_csv(r"..\Data\Source\covid_restrictions.csv")
df = df.groupby('RegionCode').StringencyIndex.agg('mean').reset_index()
df['RegionCode'] = df['RegionCode'].str.replace(r'US_', '', regex = True)
df = df.rename(columns = {'RegionCode': 'state'})
msa_states = msa_states.merge(df, on = 'state', how = 'inner')

In [4]:
msa_states.to_stata(r"..\Data\Intermediate\stringency.dta", version = 119)